<a href="https://colab.research.google.com/github/ai4ci-kpi/Refresh-in-C/blob/main/Lecture_3_File_Access_and_Pipes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



<img src="https://github.com/ai4ci-kpi/Refresh-in-C/blob/main/img/c-language-logo-simple.png?raw=true" alt="Test Image" width="100"/>



# Introduction
Today, we’ll dive into the core principles of file access and pipes in C:

1.   Understanding how to open, read, write, and close files using the standard I/O functions.
1.   Using `fseek`, `ftell`, and `rewind` to efficiently navigate within files.
1. How inter-process communication (IPC) with pipes enable data exchange between processes.
1. Creating unidirectional communication channels between parent and child processes.
1. Running external commands and capturing their output.
1. Checking for failures in `fopen`, `fread`, `pipe`, and other system calls using return values and `perror()` or `strerror()`.

Understanding file access and pipes in C is crucial because these concepts form the foundation of how C programs interact with data storage and communicate between processes.


---




# Accessing and Writing to a File

File access in C is managed using the `standard I/O library`, which provides functions to interact with files. An external file can be opened, read from, written to, and closed in a C program. For these operations, C includes the `FILE` type for defining a file stream. The file stream keeps track of where reading and writing last occurred.

The `stdio.h` library includes file handling functions: `FILE` typedef for defining a file pointer.

`FILE` is a `structure` that stores information about a file stream, including:

    

* A file descriptor
* Current position in the file
* Error and EOF indicators
* Buffering information

`fopen(filename, mode)` returns a `FILE` pointer to file `filename` which is opened using `mode`. If a file cannot be opened, `NULL` is returned.

`NULL` is a **macro** in C that represents a null pointer. It is commonly used to indicate an invalid or uninitialized pointer, especially in file handling, memory allocation, and linked data structures.

The `fopen(filename, mode)` function is used to open files in different modes. Below is a table summarizing the available modes and their behavior:

| Mode  | Description                          | File Existence Requirement | File Pointer Position | Behavior if File Exists       | Behavior if File Doesn’t Exist |
|-------|--------------------------------------|----------------------------|-----------------------|-------------------------------|---------------------------------|
| `"r"`  | Open for **reading**                 | File must exist            | Beginning of file     | Opens file for reading        | Returns `NULL`                 |
| `"w"`  | Open for **writing**                 | File may or may not exist  | Beginning of file     | Truncates file to zero length | Creates a new file             |
| `"a"`  | Open for **appending**               | File may or may not exist  | End of file           | Appends data to the end       | Creates a new file             |
| `"r+"` | Open for **reading and writing**     | File must exist            | Beginning of file     | Opens file for read/write     | Returns `NULL`                 |
| `"w+"` | Open for **reading and writing**     | File may or may not exist  | Beginning of file     | Truncates file to zero length | Creates a new file             |
| `"a+"` | Open for **reading and appending**   | File may or may not exist  | End of file           | Appends data to the end       | Creates a new file             |
| `"rb"` | Open for **reading in binary mode**  | File must exist            | Beginning of file     | Opens file for reading        | Returns `NULL`                 |
| `"wb"` | Open for **writing in binary mode**  | File may or may not exist  | Beginning of file     | Truncates file to zero length | Creates a new file             |
| `"ab"` | Open for **appending in binary mode**| File may or may not exist  | End of file           | Appends data to the end       | Creates a new file             |
| `"r+b"`| Open for **reading and writing in binary mode** | File must exist | Beginning of file | Opens file for read/write | Returns `NULL` |
| `"w+b"`| Open for **reading and writing in binary mode** | File may or may not exist | Beginning of file | Truncates file to zero length | Creates a new file |
| `"a+b"`| Open for **reading and appending in binary mode** | File may or may not exist | End of file | Appends data to the end | Creates a new file |


`fclose(fptr)` closes file opened with `FILE *fptr`, returning `0` if close was successful. `EOF` (end of file) is returned if there is an error in closing.


   
    
    



## Writing to a File

| **Function** | **Description** | **Parameters** | **Return Value** | **Key Behavior** |
|--------------|-----------------|----------------|------------------|------------------|
| `fputc()`    | Writes a single character to a file. | `int char, FILE *stream` | The character written or `EOF` on error | - Writes one character to the file.<br>- Useful for low-level character output. |
| `fputs()`    | Writes a string to a file (without a newline). | `const char *str, FILE *stream` | Non-negative value on success, `EOF` on error | - Writes a string to the file.<br>- Does not automatically add a newline. |
| `fprintf()`  | Writes formatted output to a file. | `FILE *stream, const char *format, ...` | Number of characters written or negative value on error | - Writes formatted data to the file.<br>- Similar to `printf()` but for files. |


**Example 1.** The following program opens a file for writing, writes two strings and then closes the file:

In [7]:
%%writefile file-open.c

#include <stdio.h>

int main() {
  FILE *fptr;

  // Open the file in write mode
  fptr = fopen("myfile.txt", "w");
  if (fptr == NULL) {
    printf("Error opening file.");
    return -1;  // Exit the program if the file cannot be opened
  }

  // Write to the file
  fprintf(fptr, "Hello, this is written to the file!\n");
  fputs("This is another line written using fputs.\n", fptr);

  // Close the file
  fclose(fptr);

  printf("Data written to the file successfully.\n");
  return 0;
}

Overwriting file-open.c


In [8]:
%%script bash
gcc file-open.c -o file-open
./file-open

Data written to the file successfully.


In this program, if there is an error when opening the file, a `-1` error code is returned to the system. Error handling will be explained later.

Closing a file when you are done using it is a good programming practice.

**Example 2.** Writing numerical data to a file.

This example writes an array of floating-point numbers to a file. This could represent, for example, a time series of experimental measurements.

In [11]:
%%writefile file-open2.c

#include <stdio.h>

int main() {
    // Sample data: time series of measurements
    double data[] = {1.2, 2.3, 3.4, 4.5, 5.6};
    int num_points = sizeof(data) / sizeof(data[0]);

    // Open a file for writing
    FILE *file = fopen("data.txt", "w");
    if (file == NULL) {
        printf("Error opening file!\n");
        return -1;
    }

    // Write data to the file
    for (int i = 0; i < num_points; i++) {
        fprintf(file, "%.6f\n", data[i]);  // Write each value with 6 decimal places
    }

    // Close the file
    fclose(file);

    printf("Data written to file successfully.\n");
    return 0;
}

Writing file-open2.c


In [12]:
%%script bash
gcc file-open2.c -o file-open2
./file-open2

Data written to file successfully.


# Reading from a File

The `stdio.h` library also includes functions for reading from an open file. A file can be read one character at a time or an entire string can be read into a character buffer, which is typically a char array used for temporary storage.

To read from a file you can use functions like `fscanf()`, `fgets()`, or `fread()`. Below is an example program that reads the contents of a file (`myfile.txt`) and prints it to the console.

| **Function** | **Description** | **Usage Example** | **Key Differences** |
|--------------|-----------------|-------------------|---------------------|
| `fscanf()`   | Reads formatted input from a file. | `fscanf(file, "%d %s", &num, str);` | - Reads formatted data (e.g., integers, strings).<br>- Stops at whitespace or specified delimiter. |
| `fgets()`    | Reads a string from a file until a newline or EOF is encountered. | `fgets(buffer, sizeof(buffer), file);` | - Reads a line of text.<br>- Includes the newline character in the buffer.<br>- Stops at newline or EOF. |
| `fread()`    | Reads raw data from a file into a buffer. | `fread(buffer, sizeof(char), size, file);` | - Reads binary or raw data.<br>- Does not interpret or format the data.<br>- Reads a specified number of bytes. |


**Example 1.**  Reading text files line by line.

In [9]:
%%writefile file-read.c

#include <stdio.h>

int main() {
    FILE *fptr;
    char buffer[100]; // Buffer to store the data read from the file

    // Open the file in read mode
    fptr = fopen("myfile.txt", "r");
    if (fptr == NULL) {
        printf("Error opening file.");
        return -1; // Exit the program if the file cannot be opened
    }

    // Read and print the file contents line by line
    while (fgets(buffer, sizeof(buffer), fptr) != NULL) {
        printf("%s", buffer); // Print the line to the console
    }

    // Close the file
    fclose(fptr);

    return 0;
}

Writing file-read.c


`fopen("myfile.txt", "r")`opens the file `myfile.txt` in **read mode**.

`fgets(buffer, sizeof(buffer), fptr)` reads a line from the file into the `buffer`:

* The `sizeof(buffer)` ensures that no more than `100` characters are read at a time (prevents buffer overflow).
*Returns `NULL` when the end of the file is reached.

In [10]:
%%script bash
gcc file-read.c -o file-read
./file-read

Hello, this is written to the file!
This is another line written using fputs.


**Example 2.**  Reading numerical data from a file

This example reads the data from the file stores it in an array.

In [17]:
%%writefile file-read2.c

#include <stdio.h>
#include <stdlib.h>

int main() {
    // Open the file for reading
    FILE *file = fopen("data.txt", "r");
    if (file == NULL) {
        printf("Error opening file!\n");
        return 1;
    }

    // Determine the number of data points
    int num_points = 0;
    double value;
    while (fscanf(file, "%lf", &value) == 1) {
        num_points++;
    }

    // Allocate memory for the data array
    double *data = (double *)malloc(num_points * sizeof(double));
    if (data == NULL) {
        printf("Memory allocation failed!\n");
        fclose(file);
        return 1;
    }

    // Rewind the file to read the data again
    rewind(file);

    // Read the data into the array
    for (int i = 0; i < num_points; i++) {
        fscanf(file, "%lf", &data[i]);
    }

    // Close the file
    fclose(file);

    // Print the data to verify
    printf("Data read from file:\n");
    for (int i = 0; i < num_points; i++) {
        printf("%.6f\n", data[i]);
    }

    // Free allocated memory
    free(data);

    return 0;
}

Overwriting file-read2.c


In [18]:
%%script bash
gcc file-read2.c -o file-read2
./file-read2

Data read from file:
1.200000
2.300000
3.400000
4.500000
5.600000
